<a href="https://colab.research.google.com/github/kenton-tang-kl/Stability_Prediction/blob/main/KingLongTang_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch datasets scikit-learn captum matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

In [2]:
from datasets import load_dataset

# Load the remote homology dataset from TAPE
dataset = load_dataset("proteinglm/stability_prediction")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/77.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/121k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/53614 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/2512 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12851 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['seq', 'label'],
        num_rows: 53614
    })
    valid: Dataset({
        features: ['seq', 'label'],
        num_rows: 2512
    })
    test: Dataset({
        features: ['seq', 'label'],
        num_rows: 12851
    })
})


In [3]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(10000))
dataset["valid"] = dataset["valid"].shuffle(seed=42).select(range(500))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(5000))

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['seq', 'label'],
        num_rows: 10000
    })
    valid: Dataset({
        features: ['seq', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['seq', 'label'],
        num_rows: 5000
    })
})


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("yarongef/DistilProtBert")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
def tokenize_function(example):
    return tokenizer(example["seq"],
                     padding="max_length",
                     truncation=True,
                     max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=8, shuffle=True)
val_dataloader = DataLoader(tokenized_datasets["valid"], batch_size=8)

In [7]:
import torch
import torch.nn as nn
from transformers import AutoModel

base_model = AutoModel.from_pretrained("yarongef/DistilProtBert")

pytorch_model.bin:   0%|          | 0.00/924M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at yarongef/DistilProtBert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Define a regression model using DistilProtBERT as the backbone
class ProteinStabilityRegressor(nn.Module):
    def __init__(self, base_model):
        super(ProteinStabilityRegressor, self).__init__()
        self.base_model = base_model
        # Use the hidden size from the model config (commonly 768)
        self.regressor = nn.Linear(self.base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        # Use the [CLS] token representation (first token) as the pooled output
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.regressor(cls_output)
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
        return {"loss": loss, "logits": logits}

# Initialize our model
model = ProteinStabilityRegressor(base_model)

In [14]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import mean_squared_error

# Define a compute_metrics function for evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze(-1)
    mse = mean_squared_error(labels, predictions)
    return {"mse": mse}

In [15]:
# Define training arguments: adjust parameters as needed
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,                # evaluate every 500 steps
    logging_steps=100,             # log every 100 steps
    per_device_train_batch_size=8, # adjust based on GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,            # you can increase the number of epochs
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,   # automatically load the best model when finished training
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    compute_metrics=compute_metrics,
)

In [17]:
# Start training
trainer.train()

# Evaluate on the test split after training
test_results = trainer.evaluate(tokenized_datasets["test"])
print("Test Evaluation Results:", test_results)

Step,Training Loss,Validation Loss,Mse
500,0.329500,0.449113,0.449113
1000,0.315800,0.434292,0.434292
1500,0.321000,0.422707,0.422707
2000,0.329300,0.421230,0.421230
2500,0.336100,0.426956,0.426956
3000,0.322800,0.424132,0.424132
3500,0.347600,0.425809,0.425809


Test Evaluation Results: {'eval_loss': 0.8047664761543274, 'eval_mse': 0.8047664761543274, 'eval_runtime': 271.382, 'eval_samples_per_second': 18.424, 'eval_steps_per_second': 2.303, 'epoch': 3.0}


In [18]:
model_save_path = "protein_stability_trained_model"
trainer.save_model(model_save_path)